In [1]:
# use the os
import os
import glob
# math and data structure
import numpy as np
import pandas as pd
# plotting
import matplotlib.pyplot as plt
# image processing
import cv2

In [7]:
sub = 'sub-03'

In [8]:
stim_dir = os.path.join('..','stimuli')
data_dir = os.path.join('..','rawdata')

mask_dir = os.path.join(stim_dir,'color_masks')
# forground_background masks to apply the alpha channel
fb_mask_dir = os.path.join(stim_dir, 'foreground_background_masks')

# subject directory
sub_dir = os.path.join(data_dir,sub)
# stimulus folder for subject to store subject specific equiluminant images
sub_stim_dir = os.path.join(sub_dir,'stimuli')
if not os.path.exists(sub_stim_dir):
    os.mkdir(sub_stim_dir)
sub_true_stim_dir = os.path.join(sub_stim_dir, 'true_color')
if not os.path.exists(sub_true_stim_dir):
    os.mkdir(sub_true_stim_dir)
sub_inv_stim_dir = os.path.join(sub_stim_dir, 'inverted_lab')
if not os.path.exists(sub_inv_stim_dir):
    os.mkdir(sub_inv_stim_dir)

In [9]:
# get a list of all stimuli in the directory
stimuli_full_path = glob.glob(os.path.join(sub_true_stim_dir,'*.png'))
stimuli_full_path.sort()

# extract the image name from the stimulus path
stimuli = [os.path.basename(stim) for stim in stimuli_full_path]

In [10]:
color_dict = {
    'stimuli':stimuli,
    'true_r':[],
    'true_g':[],
    'true_b':[],
    'inv_r':[],
    'inv_g':[],
    'inv_b':[]
}

for s, stim in enumerate(stimuli):
    # get path for stim
    true_path = os.path.join(sub_true_stim_dir,stim)
    inv_path = os.path.join(sub_inv_stim_dir,stim)
    # path of mask
    mask_path = os.path.join(mask_dir,stim)
    # read in stimuli
    curr_true_stim = cv2.imread(true_path)
    curr_inv_stim = cv2.imread(inv_path)
    # read in mask
    curr_mask = cv2.imread(mask_path)
    curr_mask = curr_mask[:,:,0].astype(bool)
    # get pixels of stimuli within mask
    true_pixels = curr_true_stim[curr_mask]
    inv_pixels = curr_inv_stim[curr_mask]
    # get the modus of pixel values
    true_unique, true_count = np.unique(true_pixels,axis=0,return_counts=True)
    inv_unique, inv_count = np.unique(inv_pixels,axis=0,return_counts=True)
    true_typical = true_unique[np.argmax(true_count)].astype(int)
    inv_typical = inv_unique[np.argmax(inv_count)].astype(int)
    # safe typical pixel values in dictionary
    # t_B, t_G, t_R = cv2.split(true_typical)
    # i_b, i_G, i_R = cv2.split(inv_typical)
    
    color_dict['true_r'].append(true_typical[2])
    color_dict['true_g'].append(true_typical[1])
    color_dict['true_b'].append(true_typical[0])
    
    color_dict['inv_r'].append(inv_typical[2])
    color_dict['inv_g'].append(inv_typical[1])
    color_dict['inv_b'].append(inv_typical[0])
    
# convert dictionary to dataframe
color_df = pd.DataFrame(data=color_dict,columns=color_dict.keys())
color_df.to_csv(path_or_buf=os.path.join(sub_stim_dir,'typical_colors.csv'))

In [11]:
color_df

,stimuli,true_r,true_g,true_b,inv_r,inv_g,inv_b
0,blue_nivea.png,21,51,123,52,48,8
1,blue_pool.png,3,156,213,242,98,59
2,blue_sign.png,21,72,137,87,60,18
3,green_brokkoli_1.png,77,119,66,138,97,149
4,green_frog_1.png,61,81,35,82,70,124
5,green_lettuce_1.png,104,140,35,120,121,254
6,orange_basketball.png,235,110,71,26,159,207
7,orange_carrots.png,254,151,49,0,193,254
8,orange_pumpkin.png,240,85,0,0,171,254
9,red_fire_extinguisher_1.png,203,31,31,0,95,113
